# Module 7 - Programming Assignment


# Unification

This is actually Part I of a two part assignment. In a later module, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

It really is that simple...expressions must be literally the same (identical) except if one or the other (or both) has a variable in that "spot".

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

These functions are already included in the starter .py file.

In [47]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [48]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [49]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

**Note** there was a change between 2.7 and 3.0 that "hid" the next() function in the tokenizer.

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [50]:
parse("Fred")

'Fred'

In [51]:
parse( "?x")

'?x'

In [52]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [53]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [54]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [55]:
is_variable( "Fred")

False

In [56]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [57]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [58]:
is_constant( "Fred")

True

In [59]:
is_constant( "?fred")

False

In [60]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>


You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. For an actual application, you would need to fix this!)

-----

<a id="apply"></a>
## apply

applies replacements to atomic sentences to prevent the unification algorithm from having to check the assignment history or have different variable point to the same constant

**Used by**: [unify](#unify)

* **result** dict: result of the unification of atomic sentences
* **rest_of_list1** list: list of atomic sentences 
* **rest_of_list12** list: list of atomic sentences  

* **returns** Tuple(List[str],List[Str]: updated list of atomic sentences

In [61]:
def apply(result, rest_of_list1, rest_of_list2):
    if any(key in rest_of_list1 for key in result.keys()):
        for key, value in result.items():
            for i in range(len(rest_of_list1)):
                if rest_of_list1[i] == key:
                    rest_of_list1[i] = value 
    if any(key in rest_of_list2 for key in result.keys()):
          for key, value in result.items():
            for i in range(len(rest_of_list2)):
                if rest_of_list2[i] == key:
                    rest_of_list2[i] = value
    return rest_of_list1, rest_of_list2

In [62]:
assert apply({"?x":"bob", "?y":"sussan"}, ["?x","?b","?c"], ["kevin","?x",["?x", 'father_of_kim']]) == (['bob', '?b', '?c'], ['kevin', 'bob', ['?x', 'father_of_kim']])
assert apply({"?x":"bob", "?y":"sussan"}, ["?x","?y","?c"], ["kevin","?x",["?y", 'father_of_kim']]) == (['bob', 'sussan', '?c'], ['kevin', 'bob', ['?y', 'father_of_kim']])
assert apply({"?x":"bob", "?y":"sussan"}, ["?a","?b","?c"], ["kevin","?b",["?d", 'father_of_kim']]) == (["?a","?b","?c"], ["kevin","?b",["?d", 'father_of_kim']])

In [63]:
def occurs(list_expression1, list_expression2):
    for item in list_expression1:
        if item in list_expression2 and is_variable(item):
            return True
        else:
            return False

<a id="unification"></a>
## unification

finds all instances in the knowledgebase that unify with a given sentence for the purpose of eliminating the instantation step in first order proofs 

**Used by**: [unify](#unify)

* **list_expression1** list: list of atomic sentences 
* **list_expression2** list: list of atomic sentences 


* **returns** dict: substitutions as the result unification of atomic sentences 

In [94]:
def unification(list_expression1, list_expression2):
    # base cases
    if is_constant(list_expression1) and is_constant(list_expression2) or len(list_expression1)==0 or len(list_expression2)==0:
        if list_expression1 == list_expression2:
            return {}
        else:
            return None
    if is_variable(list_expression1):
        if occurs(list_expression1, list_expression2):
            return None
        else:
            return {list_expression1:list_expression2}
    if is_variable(list_expression2):
        if occurs(list_expression2, list_expression1):
            return None
        else:
            return {list_expression2:list_expression1}
    # inductive step
    first1 = list_expression1[0]
    first2 = list_expression2[0]
    result1 = unification( first1, first2)
#     print('result1: ',result1)
    if result1 == None:
        return None
    rest_of_list1, rest_of_list2 = apply(result1, list_expression1[1:], list_expression2[1:])
#     print( rest_of_list1, rest_of_list2)
    result2 = unification(rest_of_list1, rest_of_list2)
#     print('result2: ',result2)
    if result2 == None:
        return None
#     print('comb: ',result1.update(result2))
    return result1 | result2

In [97]:
assert unification(['?x','mary'], ['bob','?y']) == {'?x': 'bob', '?y': 'mary'}
assert unification(['bob','mary'], ['bob','?y']) == { '?y': 'mary'}
assert unification(['?x','?y'], ['bob',['father', 'jeff']]) == {'?x': 'bob', '?y': ['father', 'jeff']}

In [98]:
def list_check(parsed_expression):
    if isinstance(parsed_expression, list):
        return parsed_expression
    return [parsed_expression]

The `unification` pseudocode only takes lists so we have to make sure that we only pass a list.
However, this has the side effect of making "foo" unify with ["foo"], at the start.
That's ok.

In [99]:
def unify( s_expression1, s_expression2):
    list_expression1 = list_check(parse(s_expression1))
    list_expression2 = list_check(parse(s_expression2))
    return unification( list_expression1, list_expression2)

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

## Test Cases

Use the expressions from the Self Check as your test cases...

In [121]:
self_check_test_cases = [
    ['(Fred)', '(Barney)', None],  ['(Pebbles)', '(Pebbles)', {}], 
    ['(quarry_worker Fred)', '(quarry_worker ?x)', {'?x':'Fred'}],
    ['(son Barney ?x)', '(son ?y Bam_Bam)', {'?y':'Barney', '?x':'Bam_Bam'}],
    ['(married ?x ?y)', '(married Barney Wilma)', {'?x':'Barney', '?y':'Wilma'}],
    ['(son Barney ?x)', '(son ?y (son Barney))', {'?y':'Barney', '?x':['son', 'Barney']}],
    ['(son Barney ?x)', '(son ?y (son ?y))', {'?y':'Barney', '?x':['son', '?y']}],
    ['(son Barney Bam_Bam)', '(son ?y (son Barney))', None],
    ['(loves Fred Fred)', '(loves ?x ?x)', {'?x':'Fred'} ],
    ['(future George Fred)', '(future ?y ?y)', None]
]
for case in self_check_test_cases:
    exp1, exp2, expected = case
    actual = unify(exp1, exp2)
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

actual = None
expected = None


actual = {}
expected = {}


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = {'?y': 'Barney', '?x': 'Bam_Bam'}
expected = {'?y': 'Barney', '?x': 'Bam_Bam'}


actual = {'?x': 'Barney', '?y': 'Wilma'}
expected = {'?x': 'Barney', '?y': 'Wilma'}


actual = {'?y': 'Barney', '?x': ['son', 'Barney']}
expected = {'?y': 'Barney', '?x': ['son', 'Barney']}


actual = {'?y': 'Barney', '?x': ['son', '?y']}
expected = {'?y': 'Barney', '?x': ['son', '?y']}


actual = None
expected = None


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = None
expected = None




Now add at least **five (5)** additional test cases of your own making, explaining exactly what you are testing. They should not be testing the same things as the self check test cases above.

In [131]:
new_test_cases = [
    ['(Kim Barney Megan)', '(Kim Barney Megan)', {}, "equal constants"], 
    ['(son ?x Barney)', '(Kim Barney ?x)', None, "assignment of x twice"], 
    ['(Kim Barney (?x florida))', '(Kim Barney (Pebbles (New_Jersey)))', None, "fail on constant equal to a list"], 
    ['(?x (?y ?z))', '(daughter (Wilma (Pebbles)))', {'?x':'daughter', '?y':'Wilma', '?z':['Pebbles']}, "assigmnet inside of nested list"], 
    ['(Barney ?y)', '(?x (Pennsylvania ?x))', {'?x':'Barney', '?y':['Pennsylvania', '?x']}, "looking for equiv to ['Pennsylvania', 'Barney']"]
]
for case in new_test_cases:
    exp1, exp2, expected, message = case
    actual = unify(exp1, exp2)
    print(f"Testing {message}...")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Testing equal constants...
actual = {}
expected = {}


Testing assignment of x twice...
actual = None
expected = None


Testing fail on constant equal to a list...
actual = None
expected = None


Testing assigmnet inside of nested list...
actual = {'?x': 'daughter', '?y': 'Wilma', '?z': ['Pebbles']}
expected = {'?x': 'daughter', '?y': 'Wilma', '?z': ['Pebbles']}


Testing looking for equiv to ['Pennsylvania', 'Barney']...
actual = {'?x': 'Barney', '?y': ['Pennsylvania', '?x']}
expected = {'?x': 'Barney', '?y': ['Pennsylvania', '?x']}




## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.